Implementation of DBSCAN using a K-D-Tree to improve performance

In [2]:
# imports 
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.neighbors import KDTree
from collections import deque

from sklearn import datasets
iris = datasets.load_iris()

In [14]:
class DBSCAN:
    
    def __init__(self, points, max_dist, min_points):
        
        self.points = points
        self.max_dist = max_dist
        self.min_points = min_points + 1
        self.tree = KDTree(points)
        self.lookup = defaultdict(lambda:None)
    
    # iterate through points, when it has a label continue, otherwise query for neighbours, if it is core start expanding the cluster, 
    # when cluster is done increase label counter by one
    
    def range_query(self,point):
        
        index, = self.tree.query_radius([point], self.max_dist)
        
        return self.points[index]
           
    def fit(self):
        
        label = 0
        
        for point in self.points:
            
            # if point already has a label continue
            if self.lookup[tuple(point)] != None:
                
                continue
                
            # count neighbors
            neighbors = self.range_query(point) 
            
            # if point has less than min_points as neighbors classify as noise
            if len(neighbors) < self.min_points:
                
                self.lookup[tuple(point)] = -1
                
            # otherwise point is core and the cluster needs to be expanded from there
            else:
                
                # set label
                self.lookup[tuple(point)] = label
                
                # add all neighbors to a queue, the cluster will be expanded using bft
                queue = deque()
                
                for neighbor in neighbors:
                    
                    queue.appendleft(neighbor)
                
                while len(queue) > 0:
                    
                    next_point = queue.pop()
                    
                    # if next point was marked as noise it is now part of a cluster but not a core
                    if self.lookup[tuple(next_point)] == -1:
                        
                        self.lookup[tuple(next_point)] = label
                        continue
                    
                    # if point already has a label it was already assigned a cluster
                    if self.lookup[tuple(next_point)] is not None:
                        
                        continue
                    
                    # if next point had no label it gets assigned to the current cluster
                    self.lookup[tuple(next_point)] = label
                    
                    # now we need to check if next point is core too
                    next_neighbors = self.range_query(next_point) 
                    
                    # if next point is not core nothing else needs to be done
                    if len(next_neighbors) < self.min_points:
                        
                        continue
                    
                    # otherwise all neighbors of next core need to be checked too
                    else:
                        
                        for next_neighbor in next_neighbors:
                            
                            queue.appendleft(next_neighbor)
                            
                # when cluster is expanded to its limits increase label by 1
                label += 1
                            

In [ ]:
test = DBSCAN(iris.data, 0.5, 5)
test.fit()
test.lookup